### 1. Importar librerías para abrir coordenadas en archivos de polígonos .shx y .shp:

In [43]:
import geopandas as gpd                                             # leer los archivos .shx y .shp
from shapely.geometry import Polygon, MultiPolygon, shape           # Simplificar Polígonos
from shapely.ops import unary_union                                 # Combinar polígonos

In [44]:
# Cargar los datos geoespaciales
data = gpd.read_file('../Datasets/taxi_zones.shx')

In [45]:
data

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


para poder leer el archivo taxi_zones.shx en la misma carpeta debe estar el arcivo taxi_zones.shp y taxi_zones.dbf

### 2. Transformar coordenadas planas ATEST WELL-KNOWN IDENTIFIER 2263 a la proyección inversa mundial (EPSG:4326):

In [46]:
gdf = gpd.GeoDataFrame(data)

# Definir EPSG:2263 que está en pies (LATEST WELL-KNOWN IDENTIFIER 2263):
gdf.crs = 'EPSG:2263'

# Corregir coordenadas planas a mundiales:
gdf_latlon = gdf.to_crs('EPSG:4326')  # Proyección inversa al sistema de coordenadas geográficas

# Calcular los centroides de los polígonos
gdf_latlon['centroid'] = gdf_latlon['geometry'].centroid

# Extraer las coordenadas x e y de los centroides
gdf_latlon['centroid_x'] = gdf_latlon['centroid'].x
gdf_latlon['centroid_y'] = gdf_latlon['centroid'].y

C:\Users\davin\AppData\Local\Temp\ipykernel_948\804222979.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_latlon['centroid'] = gdf_latlon['geometry'].centroid


### 3. Ordenar y selecionar columnas:

In [47]:
# Seleccionar las columnas deseadas
gdf_latlon = gdf_latlon[['OBJECTID', 'borough', 'zone', 'centroid_x', 'centroid_y','geometry']]

# Renombrar las columnas centroid_x y centroid_y a longitud y latitud respectivamente
gdf_latlon = gdf_latlon.rename(columns={'OBJECTID':'LocationID','centroid_x': 'longitud', 'centroid_y': 'latitud'})

### 4. Simplificar polígonos:

In [48]:
# Definir la tolerancia para la simplificación
tolerancia = 0.0001  # ajusta este valor según sea necesario

# Función para simplificar una geometría individual
def simplificar_geometria(geom, tolerancia):
    return geom.simplify(tolerancia)

# Aplicar la función de simplificación a cada geometría en el GeoDataFrame
gdf_latlon['geometry'] = gdf_latlon['geometry'].apply(lambda geom: simplificar_geometria(geom, tolerancia))

### 5. Eliminar datos duplicados unificando los polígonos de los ID 55+56 y 103+104+105:

In [49]:
# Seleccionar los polígonos correspondientes a los LocationID 56 y 57
poligonos_56_57 = gdf_latlon.loc[gdf_latlon['LocationID'].isin([56, 57]), 'geometry']

# Crear un MultiPolygon combinando los polígonos
multi_poligono_56_57 = unary_union(poligonos_56_57)

# Reemplazar el polígono del LocationID 56 con el MultiPolygon combinado
gdf_latlon.loc[gdf_latlon['LocationID'] == 56, 'geometry'] = multi_poligono_56_57

# Eliminar el registro correspondiente al LocationID 57
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 57]

In [50]:
# Seleccionar los polígonos correspondientes a los LocationID 103, 104 y 105
poligonos_103_104_105 = gdf_latlon.loc[gdf_latlon['LocationID'].isin([103, 104, 105]), 'geometry']

# Crear un MultiPolygon combinando los polígonos
multi_poligono_103_104_105 = unary_union(poligonos_103_104_105)

# Reemplazar el polígono del LocationID 103 con el MultiPolygon combinado
gdf_latlon.loc[gdf_latlon['LocationID'] == 103, 'geometry'] = multi_poligono_103_104_105

# Eliminar el registro correspondiente al LocationID 104 y 105
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 104]
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 105]

Se guarda archivo como Taxi_Zone.csv en DataClean